In [3]:
import pandas as pd
import numpy as np

# Correlation vs Causation
Simpsons Paradox in the number of students entering majors in UC Berkeley
Another example of lack of insight, is the confusion of correlation and causation

Let's see an example where we may be tempted to reach the wrong conclusion

*I am reminded* that we discussed the "Admissions Case Study" in Lesson 7, for which I didn't make a note?
https://classroom.udacity.com/courses/st101/lessons/48759015/concepts/487550470923

Oh yes I do have notes, they're just in the first section 
http://localhost:8888/notebooks/udacity_notes/st101__1_Visualizing_relationships_in_data__7_Admissions%20Case%20Study.ipynb

I am reminded of the data for Simpsons paradox. In each of their Major, females  had higher rates of admission as a proportion of female applicants. But in total, (i.e. across all majors; ..sum() the panel...) , women were accepted at a lower rate. 


In [8]:
def admissions_3_panel():
    return pd.Panel({'Male': pd.DataFrame({
                    'Applied': pd.Series({'Major A': 900, 'Major B':100}), 
                    'Admitted': pd.Series({'Major A' : 450, 'Major B': 10})
    }),
                 'Female': pd.DataFrame({
                     'Applied': pd.Series({'Major A' : 100, 'Major B': 900}),
                     'Admitted': pd.Series({ 'Major A': 80, 'Major B': 180})
                 })})
    return panel
admissions_3_panel().sum()

,Female,Male
Admitted,260,460
Applied,1000,1000


In [11]:
#There is no need to compare the denominator; the number of those who applied
#in each gender, since those numbers are the same.
#Lets just consider the numerator; look how many more men were admitted than women;
admissions_3_panel().sum().loc['Admitted']
# No need to consider denom / admissions_3_panel().sum().loc['Applied']

Female    260
Male      460
Name: Admitted, dtype: int64

In the above case; we needed to *aggregate* (i.e. sum up *across* Majors) to see the true effect of lower-admission-rates-for-women.
In this case we look at a similar phenomenon , but it goes the other way; we gain insight by *subdividing* our population into those who are sick and those who are healthy ( a m. keep this in mind for when Sebastian distinguishes between 'Sick' and 'Healthy'

In [23]:
hospital_vs_home = pd.DataFrame(index=('total', 'died'), data={'in hospital': [40, 4], 'at home': [8000, 20]}, columns=['in hospital', 'at home'])

In [24]:
hospital_vs_home

,in hospital,at home
total,40,8000
died,4,20


# Indexing in Pandas

I keep getting confused on this, hopefully this helps me remember what to avoid:
## Don't use `ix`, use `loc` or `iloc`
## Don't use `(...)`, use `[ indexer ]`

In [25]:
hospital_vs_home.loc['died']/ hospital_vs_home.loc['total']

in hospital    0.1000
at home        0.0025
dtype: float64

In [36]:

def quiz_mortality():
    """Compute for me the percentages of the people who died -- in the hospital, and at home"""
    return (hospital_vs_home.loc['died']/ hospital_vs_home.loc['total'])

In [38]:
actual_values = quiz_mortality().values
expected_values =  [0.1000, 0.0025]
assertion_msg = 'Expect the [hospital death rate, at-home death rate] to be {} , but actual values were {}'.format(expected_values, actual_values)
assert np.allclose(actual_values, expected_values), assertion_msg


# Chances
Chances of dying in a hospital are 40 times larger than the chances of dying at home?
This is an illustration of correlation
## Correlation
See next example; 40 times larger:

In [39]:
quiz_mortality()['in hospital'] / quiz_mortality()['at home']

40.0

In [40]:
def quiz_deciding():
    """As a smart statistics student, knowing your chances of dying in the hospital are higher, will you stay at home"""
    #I cannot explain why I should answer False, except Sebastian seems to be setting us up to expect that:
    return False

# Difference in  Language: Correlation vs Causation
## Correlation Statement

 > Chances of dying in a hospital are 40 times larger than at home

## Causation statement

 > Being in a hospital --and being in a hospital alone-- increaes your probability of dying by a factor of 40

I'll admit I have a hard time understanding the semantic difference; can we get more precise language to describe it?